## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [4]:
(x, y), (x_tesy, y_test)=mnist_dataset()
print(x.shape, y.shape, x.min(), x.max())

(60000, 28, 28) (60000,) 0.0 1.0


## 建立模型

In [5]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28*28, 120], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 120],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[120], dtype=tf.float32, initial_value=tf.zeros(120))
        self.W2 = tf.Variable(shape=[120, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[120, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        reshaped_input = tf.reshape(x, shape=[-1, 28*28])
        hidden_layer = tf.tanh(tf.matmul(reshaped_input, self.W1) + self.b1)
        logits = tf.matmul(hidden_layer, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [6]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [8]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.0471516 ; accuracy 0.43673334
epoch 1 : loss 2.0421503 ; accuracy 0.44261667
epoch 2 : loss 2.0371652 ; accuracy 0.44856668
epoch 3 : loss 2.0321953 ; accuracy 0.4538
epoch 4 : loss 2.0272412 ; accuracy 0.45986667
epoch 5 : loss 2.0223022 ; accuracy 0.46568334
epoch 6 : loss 2.017378 ; accuracy 0.47065
epoch 7 : loss 2.012469 ; accuracy 0.47591665
epoch 8 : loss 2.007574 ; accuracy 0.48111665
epoch 9 : loss 2.0026937 ; accuracy 0.48618335
epoch 10 : loss 1.9978275 ; accuracy 0.49111667
epoch 11 : loss 1.9929752 ; accuracy 0.4955
epoch 12 : loss 1.9881368 ; accuracy 0.5002667
epoch 13 : loss 1.9833118 ; accuracy 0.5047
epoch 14 : loss 1.9785002 ; accuracy 0.50941664
epoch 15 : loss 1.9737021 ; accuracy 0.514
epoch 16 : loss 1.9689168 ; accuracy 0.5181
epoch 17 : loss 1.9641448 ; accuracy 0.52216667
epoch 18 : loss 1.9593854 ; accuracy 0.52603334
epoch 19 : loss 1.9546388 ; accuracy 0.5301667
epoch 20 : loss 1.9499047 ; accuracy 0.53426665
epoch 21 : loss 1.945183 ; accu